##Importing Libraries

In [17]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

##Generating Linear separable data

In [18]:
X, y = make_classification(
    n_samples=500,
    n_features=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=2.0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


##Perceptron class for linear seperable data

In [19]:
class Perceptron:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros(n_features)
        self.b = 0

        for _ in range(self.epochs):
            for i in range(n_samples):
                linear_output = np.dot(X[i], self.W) + self.b
                y_pred = 1 if linear_output >= 0 else 0


                self.W += self.lr * (y[i] - y_pred) * X[i]
                self.b += self.lr * (y[i] - y_pred)

    def predict(self, X):
        linear_output = np.dot(X, self.W) + self.b
        return np.where(linear_output >= 0, 1, 0)


perceptron = Perceptron()
perceptron.fit(X_train, y_train)


##Accuracy prediction for single perceptron on linear seperable data

In [20]:
y_pred = perceptron.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print("Perceptron Accuracy:", accuracy)

Perceptron Accuracy: 0.99


##Testing single perceptron on Non-Linear seperable data

In [21]:
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=500, noise=0.1, factor=0.5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

perceptron1 = Perceptron()
perceptron1.fit(X_train, y_train)

y_pred = perceptron1.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print("Perceptron1 Accuracy:", accuracy)

Perceptron1 Accuracy: 0.56


##Activation Functions

In [22]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


##Multi-layer perceptron class

In [23]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, lr=0.01):
        self.lr = lr


        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)

        self.W2 = np.random.randn(hidden_size, 1)
        self.b2 = np.zeros(1)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = relu(self.z1)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)

        return self.a2

    def backward(self, X, y):
        m = X.shape[0]

        #output layer
        dz2 = self.a2 - y.reshape(-1, 1)
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0) / m

        #Hidden layer
        dz1 = np.dot(dz2, self.W2.T) * relu_derivative(self.z1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0) / m

        #weights updation
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=1000):
        for _ in range(epochs):
            self.forward(X)
            self.backward(X, y)

    def predict(self, X):
        probs = self.forward(X)
        return (probs >= 0.5).astype(int)


##Training and evaluating accuracy for non-linear data

In [24]:

X, y = make_circles(n_samples=500, noise=0.1, factor=0.5)

nn = NeuralNetwork(input_size=2, hidden_size=10, lr=0.1)
nn.train(X, y, epochs=2000)

# Evaluation
predictions = nn.predict(X)
accuracy = np.mean(predictions.flatten() == y)
print("Neural Network Accuracy:", accuracy)


Neural Network Accuracy: 0.99
